In [1]:
import pandas as pd

In [3]:
data_all =[]
for i in range(4):
    data= pd.read_csv('/Users/apple/python/SkillFactory/Ml_learning/9module/archive/combined_data_' + str(i+1) + '.txt', header=None,names = ['Cust_Id', 'Rating'], usecols = [0,1])
    data_all.append(data)

In [4]:
df = pd.concat(data_all)

In [24]:
import numpy as np
df.index = np.arange(0,len(df))
print('Full dataset shape: {}'.format(df.shape))
print('-Dataset examples-')
print(df.iloc[::5000000, :])
df.Movie_Id.nunique()

Full dataset shape: (100480507, 3)
-Dataset examples-
           Cust_Id  Rating  Movie_Id
0          1488844     3.0         1
5000000     501954     2.0       996
10000000    404654     5.0      1962
15000000    886608     2.0      2876
20000000   1193835     2.0      3825
25000000   1899206     3.0      4661
30000000    154804     4.0      5496
35000000   2078749     5.0      6274
40000000    450763     5.0      7057
45000000    102092     3.0      7991
50000000    220298     5.0      9023
55000000    550530     5.0     10042
60000000    222570     3.0     11038
65000000   1273080     5.0     11875
70000000   2026970     5.0     12676
75000000    506044     4.0     13582
80000000    353605     2.0     14453
85000000    664606     3.0     15116
90000000   2213715     3.0     16008
95000000   1589401     5.0     16879
100000000  2314006     4.0     17627


17770

In [6]:
df.shape


(100498277, 2)

In [26]:
cust_sample= df.Cust_Id.sample(10000)

In [27]:
movie_sample = df.Movie_Id.sample(5000)

In [7]:
import numpy as np
df_nan = pd.DataFrame(pd.isnull(df.Rating))
df_nan = df_nan[df_nan['Rating'] == True]
df_nan = df_nan.reset_index()

movie_np = []
movie_id = 1

for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,np.abs(i-j-1)), movie_id)
    movie_np = np.append(movie_np, temp)
    movie_id += 1

# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)

In [8]:
df = df[pd.notnull(df['Rating'])]

df['Movie_Id'] = movie_np.astype(int)
df['Cust_Id'] = df['Cust_Id'].astype(int)
print('-Dataset examples-')
print(df.sample(3))

-Dataset examples-
          Cust_Id  Rating  Movie_Id
6465411    390445     4.0      1267
18475848  1779412     4.0      3538
26306613  2300041     3.0      4884


In [11]:
df.head()

,Cust_Id,Rating,Movie_Id
1,1488844,3.0,1
2,822109,5.0,1
3,885013,4.0,1
4,30878,4.0,1
5,823519,3.0,1


In [12]:
good = df[df['Rating']==5].groupby('Cust_Id')['Movie_Id'].apply(lambda r: ' '.join([str(A) for A in r]))

In [13]:
good.head()

Cust_Id
6     175 457 886 1467 2372 2452 2782 3290 4043 4633...
7     8 30 83 175 257 283 285 313 357 457 458 468 50...
8     1202 1799 1905 2186 3610 3925 4306 5054 5317 5...
10    473 985 1542 1905 2172 3124 3371 3962 4043 430...
25                4432 6786 7605 9326 10643 15107 15270
Name: Movie_Id, dtype: object

In [15]:
import apyori

In [16]:
association_rules = apyori.apriori(good.apply(lambda r: r.split(' ')), 
                                   min_support=0.04, 
                                   min_confidence=0.1, min_lift=2, 
                                   min_length=2)

In [17]:
association_rules

<generator object apriori at 0x3649d4430>

In [18]:
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])
for item in association_rules:
    pair = item[0] 
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][0])), \
                                  ' '.join(list(item[2][0][1])),\
                                  item[2][0][2], item[1], item[2][0][3]

In [19]:
asr_df

,from,to,confidence,support,lift
0,10042,10820,0.266215,0.041573,4.110222
1,10042,10947,0.351478,0.054888,2.421943
2,10042,11064,0.348135,0.054366,2.300702
3,10042,11089,0.331616,0.051786,2.488266
4,10042,11283,0.454613,0.070994,2.206234
...,...,...,...,...,...
2018,11521,16265 2452 7057 7230 14961 5582,0.199233,0.041485,4.630662
2019,11521,2452 7057 7230 9628 14961 5582,0.195898,0.04079,4.63469
2020,14240,16265 2452 7057 7230 14961 5582,0.208372,0.041711,4.843073
2021,14240,2452 7057 7230 9628 14961 5582,0.204569,0.04095,4.839831


In [20]:
asr_df.sort_values('lift').tail(10)

,from,to,confidence,support,lift
1881,14961,16265 2452 7057 7230,0.375413,0.046806,7.368337
2000,14961,16265 7057 7230 9628 5582,0.330554,0.041213,7.380208
1999,14961,16265 2452 7057 7230 5582,0.345086,0.043025,7.395181
325,14454,457,0.631952,0.060757,7.406606
1187,7057,9628 7230,0.449937,0.056266,7.434794
143,11443,17627,0.734164,0.065722,7.958336
768,11443,2452 17627,0.463653,0.041506,8.6031
764,11443,11521 17627,0.465292,0.041653,8.756512
766,11443,12338 17627,0.490663,0.043924,9.478307
313,14302,16147,0.741323,0.040359,12.719342
